<a href="https://colab.research.google.com/github/HDWilliams/FashionMNISTClassifier/blob/master/FashionMNISTClassifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#needed if running on local machine. This project was made using Colab which does not require this step
#!pip install torch torchvision


In [0]:
#import relevant dependancies
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

from torchvision import datasets, transforms

#set up transform for a black and white image i.e. size = [1, #, #]
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

#set up trainset and trainloader and apply transform, do same for testset and testloader
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


In [0]:
#Set up Neural Network
import torch.nn as nn
import torch.nn.functional as F

class FashionClassifier(nn.Module):
  #envoke __init__ method of parent class
  def __init__(self):
    super().__init__()
    
    #define layers of network, 2 hidden layers and one output layer. All layers are linear
    #input of the next layer will be equal to the output of the previous layer as all layers are fully connected
    #output layer must have same number of nodes as classes in the dataset
    
    inp = 784 # 28 X 28 image
    hid1 = 128 
    hid2 = 64
    out = 10 # 10 classes of clothing
    self.layer1 = nn.Linear(inp, hid1)
    self.layer2 = nn.Linear(hid1, hid2)
    self.outputLayer = nn.Linear(hid2, out)
  
  def forward(self, x):
    x = F.relu(self.layer1(x))
    x = F.relu(self.layer2(x))
    
    log_softmax = nn.LogSoftmax(dim=1)
    output = log_softmax(self.outputLayer(x))
    return output

model = FashionClassifier()


In [0]:
#set up optimizer for SGD and criterion for loss function
#loss function will be Negative Log Likelihood for multiclass classification 

#learning rate = lr
lr = .01

criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [0]:
#train the model
epochs = 5
loss_by_epoch = []
for epoch in range(epochs):
  running_loss = 0
  for images, labels in trainloader:
    #reshape images into expected 2D
    images = images.view(images.shape[0], -1)
    optimizer.zero_grad()
    
    
    output = model(images)
    loss = criterion(output, labels)
    
    #calculate gradients and backpropagate 
    loss.backward()
    
    #take step using SGD
    optimizer.step()
    
    running_loss += loss.item()
  #print loss every epoch
  loss_by_epoch.append(running_loss)
  print ('Loss: ', running_loss / len(trainloader))
print(loss_by_epoch)
    
    
    
  

In [0]:
#test Network

#Visualization help functions from GitHub: https://github.com/iArunava/Intro-to-Deep-Learning-with-Pytorch-Udacity-Solutions/blob/master/intro-to-pytorch/Part%204%20-%20Fashion-MNIST.ipynb
def view_classify(img, ps, version="MNIST"):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    if version == "MNIST":
        ax2.set_yticklabels(np.arange(10))
    elif version == "Fashion":
        ax2.set_yticklabels(['T-shirt/top',
                            'Trouser',
                            'Pullover',
                            'Dress',
                            'Coat',
                            'Sandal',
                            'Shirt',
                            'Sneaker',
                            'Bag',
                            'Ankle Boot'], size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()
    

def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

### end of visulaization helper functions

def viewNumImg(numImgs):
  data = iter(testloader)
  images, labels = data.next()

  for img in images[0:numImgs]:
    # Convert 2D image to 1D vector
    img = img.view(1, -1)

    # TODO: Calculate the class probabilities (softmax) for img
    ps = torch.exp(model(img))
    print (ps)

    # Plot the image and probabilities
    #helper.view_classify(img.resize_(1, 28, 28), ps, version='Fashion')
    view_classify(img.resize_(1, 28, 28), ps, version='Fashion')

viewNumImg(3)